In [35]:
import os, glob

In [36]:
import glob
import random
import tqdm

In [37]:
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image

In [38]:
import tensorflow as tf
tf.__version__

'2.14.0'

In [39]:
%cd /content/drive/MyDrive/Datasets/MIPI_tetra/train

/content/drive/MyDrive/Datasets/MIPI_tetra/train


In [40]:
gt_files = glob.glob("gt/*.png")
input_files = glob.glob("input/*.npy")
gt_files.sort()
input_files.sort()

In [41]:
print(len(gt_files), len(input_files))

800 800


In [ ]:
!mkdir pairs2

In [ ]:
%ls -la

total 5380096
drwx------ 2 root root       4096 Jan 23 10:39 gt/
-rw------- 1 root root 3531585291 Jan 23 01:55 gt.zip
-rw------- 1 root root    2851880 Jan 23 01:49 hybirdevs_demosaic_start_here.zip
drwx------ 2 root root       4096 Jan 23 10:42 input/
-rw------- 1 root root 1974760586 Jan 23 01:52 input.zip
drwx------ 2 root root       4096 Jan 23 13:13 .ipynb_checkpoints/
drwx------ 2 root root       4096 Jan 23 14:57 pairs/
-rw------- 1 root root       2298 Jan 23 01:49 scoring_program.zip


# Image Slicing

In [ ]:
resol = 128
for idx,  (gfile, ifile) in enumerate(zip(gt_files, input_files)):
    if idx<320:
         continue

    print(idx, gfile, ifile)

    gt_img = np.array(Image.open(gfile))
    in_img = np.load(ifile)

    print(type(gt_img), gt_img.shape, type(in_img), in_img.shape)


    ny = np.ceil(in_img.shape[0] / resol ).astype(int)
    nx = np.ceil(in_img.shape[1] / resol ).astype(int)

    out_folder = 'pairs2'

    name = gfile.split('/')[-1][:-4]
    num = 0
    print(name, )
    for y in range(ny):
        for x in range(nx):
            sy, ey = y*resol, (y+1)*resol
            sx, ex = x*resol, (x+1)*resol

            if (ey>in_img.shape[0]):
                ey = in_img.shape[0]
                sy = ey-resol
            if (ex>in_img.shape[1]):
                ex = in_img.shape[1]
                sx = ex-resol

            assert resol == ey-sy
            assert resol == ex-sx

            gt_patch = gt_img[sy:ey, sx:ex]
            in_patch = in_img[sy:ey, sx:ex]
            # print(gt_patch.shape, in_patch.shape)

            seq = "_%04d" % num

            name_seq_gt = name + seq + '_gt.npy'
            name_seq_in = name + seq + '_in.npy'


            name_seq_gt = os.path.join(out_folder,name_seq_gt)
            name_seq_in = os.path.join(out_folder,name_seq_in)
            # print(name_seq_gt, name_seq_in)

            with open(name_seq_gt, 'wb') as f:
                np.save(f, gt_patch)
            with open(name_seq_in, 'wb') as f:
                np.save(f, in_patch)

            num += 1

    # break

320 gt/0321.png input/0321.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0321
321 gt/0322.png input/0322.npy
<class 'numpy.ndarray'> (1644, 2040, 3) <class 'numpy.ndarray'> (1644, 2040)
0322
322 gt/0323.png input/0323.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0323
323 gt/0324.png input/0324.npy
<class 'numpy.ndarray'> (2040, 1356, 3) <class 'numpy.ndarray'> (2040, 1356)
0324
324 gt/0325.png input/0325.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0325
325 gt/0326.png input/0326.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0326
326 gt/0327.png input/0327.npy
<class 'numpy.ndarray'> (1152, 2040, 3) <class 'numpy.ndarray'> (1152, 2040)
0327
327 gt/0328.png input/0328.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0328
328 gt/0329.png input/0329.npy
<class 'numpy.ndarray'> (2040, 1356, 3) <class 'numpy.ndarray'> (

In [ ]:
!ls -la pairs2

In [42]:
%pwd

'/content/drive/MyDrive/Datasets/MIPI_tetra/train'

In [43]:
%ls -la

total 5380100
drwx------ 2 root root       4096 Jan 23 10:39 gt/
-rw------- 1 root root 3531585291 Jan 23 01:55 gt.zip
-rw------- 1 root root    2851880 Jan 23 01:49 hybirdevs_demosaic_start_here.zip
drwx------ 2 root root       4096 Jan 23 10:42 input/
-rw------- 1 root root 1974760586 Jan 23 01:52 input.zip
drwx------ 2 root root       4096 Jan 23 13:13 .ipynb_checkpoints/
drwx------ 2 root root       4096 Jan 23 14:57 pairs/
drwx------ 2 root root       4096 Jan 24 01:57 pairs2/
-rw------- 1 root root       2298 Jan 23 01:49 scoring_program.zip


In [44]:
%cp pairs2/* pairs/

/bin/bash: line 1: /usr/bin/cp: Argument list too long


In [45]:
f1 = glob.glob('pairs/*.npy')
f2 = glob.glob('pairs2/*.npy')

In [ ]:
print(len(f1), len(f2))

# shuffle patch, order is in "order" array

In [ ]:
pnames_gt = glob.glob(os.path.join("pairs", "*_gt.npy"))
pnames_in = glob.glob(os.path.join("pairs", "*_in.npy"))

pnames_gt.sort()
pnames_in.sort()

flen = len(pnames_gt)
print(flen)


order = np.arange(flen)
np.random.shuffle(order)
print(order)



pnames_gt = [ pnames_gt[x] for x in order]
pnames_in = [ pnames_in[x] for x in order]


In [ ]:
for idx,  (gfile, ifile) in enumerate(zip(gt_files, input_files)):
    # print(idx, gfile, ifile)


In [ ]:
def write_tfrecords(file_list, idx_split, dataset_name, output_path):

    ext = file_list[0].split('.')[-1]

    tfrecords_name = '%s_%03d.tfrecords' % (dataset_name, idx_split)
    tfrecords_name = os.path.join(output_path, tfrecords_name)

    if os.path.isfile(tfrecords_name):
        print('OHNO -->. tfrecord (%s) exists' % tfrecords_name)
        #return

    print('tfrecord not exists')
    print('--->', tfrecords_name)

    writer = tf.io.TFRecordWriter(tfrecords_name)

    for index, file_name in enumerate(tqdm.tqdm(file_list)):
#         print('%d / %d' % (index+1, len(file_list)))


        # for png
        if ext == 'npy':
            image_arr = np.load(file_name)
        else:
            mage = Image.open(file_name)
            image_arr = np.array(image)

        image_byte = image_arr.tobytes()

        example = tf.train.Example(
                features=tf.train.Features(feature={
                    'image': tf.train.Feature(
                            bytes_list=tf.train.BytesList(value=[image_byte])
                    )
                })
        )



        writer.write(example.SerializeToString())
    writer.close()

In [ ]:
def make_tfrecords(flist, num_splits, dataset_name, output_path='.'):



    assert len(flist) > 0, f'no file is found'
    random.shuffle(flist)
    random.shuffle(flist)

    total_len = len(flist)
    split_len = total_len // num_splits

    print(total_len, split_len)
    start_idx = np.array(list(range(num_splits)) )* split_len
    finish_idx= (np.array((list(range(num_splits)) )) + 1) * split_len

    print(start_idx)
    print(finish_idx)
    finish_idx[-1] = total_len

    # exit()
    # print(start_idx)
    # print(finish_idx)

    for idx_split in range(num_splits):
        print(f'... {idx_split+1} / {num_splits}')
        idx_s = start_idx[idx_split]
        idx_f = finish_idx[idx_split]
        write_tfrecords(flist[idx_s:idx_f], idx_split, dataset_name, output_path)

    print('done make_tfrecords')

In [ ]:
## make tfrecords
# num_of_splits = 10
# make_tfrecords(flist=files, num_splits=num_of_splits, dataset_name='mit10_1_cyclegan_old_train', output_path='tfrecords')

In [ ]:
2480/128, 1920/128

(19.375, 15.0)

In [ ]:
20*20

400